<a href="https://www.kaggle.com/code/yaaangzhou/playground-s3-e21-very-simple-approach?scriptVersionId=141764925" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Based on public kernal. thanks to all for your efforts.

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('/kaggle/input/playground-series-s3e21/sample_submission.csv')

In [3]:
target = 'target'
num_var = df.columns.drop(['target','id'])

In [4]:
df = df[df.id != 2365]
df.target = df.target.clip(7, 20)

In [5]:
# SGD
clf = SGDOneClassSVM(nu=0.55).fit(df.drop(['id'],axis=1))
df.drop(np.array(np.where(clf.predict(df.drop(['id'],axis=1))==-1))[0], axis=0, inplace=True)
df = df.reset_index(drop=True)

# IsolationForest
lf = IsolationForest(random_state=0).fit(df.drop(['id'],axis=1))
df.drop(np.array(np.where(lf.predict(df.drop(['id'],axis=1))==-1))[0], axis=0, inplace=True)
df = df.reset_index(drop=True)

# LOF
lof = LocalOutlierFactor(n_neighbors=3, contamination=0.1)
predictions = lof.fit_predict(df[num_var])
df = df.drop(index=np.where(predictions == -1)[0])
df.reset_index(drop=True, inplace=True)

# Tricks from https://www.kaggle.com/code/warcoder/lb-1-32253-lof-svm-iforest-cleanlab
lowest_quality_labels = [2294,448, 437,309,1684,773,1722,2007]
df.drop(list(lowest_quality_labels),inplace=True)

In [6]:
drop_index = np.random.choice(df[df['target']==7].index.to_numpy(), 130, replace=False)
df.loc[drop_index]

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
2188,2499,7.0,5.90,6.100,9.110,6.480,7.300,11.24,6.94,0.210,...,14.775,2.710,0.567,4.80,3.30,3.630,8.250,7.750,3.000,3.17
2163,2466,7.0,7.40,8.500,8.420,6.650,5.015,8.76,8.11,0.473,...,5.070,1.730,0.760,4.80,3.25,2.800,5.825,30.050,5.025,5.20
1884,2150,7.0,5.95,8.433,9.545,10.080,9.250,10.40,7.20,0.247,...,14.775,0.636,1.800,4.20,3.15,10.665,10.465,8.400,5.075,7.37
1955,2234,7.0,5.10,6.500,8.880,7.170,5.020,10.07,5.07,0.490,...,7.450,1.570,1.795,4.30,3.40,4.105,5.725,5.790,3.600,4.14
2842,3232,7.0,7.10,7.300,9.500,6.100,9.250,9.11,7.90,0.200,...,3.660,1.830,2.670,3.90,4.80,4.500,6.200,6.750,3.600,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,331,7.0,5.00,5.400,9.265,9.265,9.265,10.20,6.37,0.205,...,7.795,3.730,3.730,4.70,4.80,2.025,10.375,5.725,2.210,4.40
962,1103,7.0,5.89,9.000,8.770,5.240,4.660,8.43,7.15,0.240,...,7.795,0.636,1.800,3.15,2.52,4.725,6.075,6.750,4.725,4.14
1423,1626,7.0,5.90,5.200,8.070,7.170,5.200,8.11,6.43,0.280,...,7.690,0.567,0.800,5.10,4.90,2.800,5.200,8.400,2.300,3.60
2604,2964,7.0,7.17,8.200,9.500,7.600,6.300,9.43,5.03,0.240,...,5.760,4.320,4.720,5.85,3.16,4.725,6.075,9.250,5.750,5.20


In [7]:
col_to_keep = ['O2_1', 'O2_2', 'BOD5_5', 'target']

for i in df.columns:
    if i not in col_to_keep:
        df[i]=0

In [8]:
df.to_csv('submission.csv',index=False)
df

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
0,0,8.59,7.500,9.000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,16.645,0,0
1,0,9.10,13.533,40.900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5.725,0,0
2,0,8.21,3.710,5.420,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6.750,0,0
3,0,8.39,8.700,8.100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.670,0,0
4,0,8.07,8.050,8.650,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.400,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072,0,8.65,8.367,8.760,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.415,0,0
3073,0,8.08,6.250,8.300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7.695,0,0
3074,0,8.09,6.630,6.630,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.415,0,0
3075,0,9.95,8.367,8.433,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.400,0,0
